Given a polymer structure file, a researcher interested in running moecular dynamics simulations will be highly interested  
in parameterizing this structure and using it to generate MD input files for their dynamics engine of choice.   

Here I show how to do this with polymerist toolks and the extremely useful [OpenFF Toolkit](https://docs.openforcefield.org/projects/toolkit/en/stable/)


We opt for the "here's one I made earlier" approach taken by cooking shows and provide you with some pre-made polymer structure and repeat units fragments  
However, you are highly encouraged to peruse the [polymerization demos](../1-polymerization/0-index.ipynb) if you haven't already, as these teach you how to make your own from scratch  

In [ ]:
from pathlib import Path

from openff.toolkit import Topology
from polymerist.polymers.monomers import MonomerGroup
from polymerist.mdtools.openfftools.partition import partition


example_dir = Path('pnipaam_example')
fragment_file  = example_dir / 'pnipaam.json'
structure_file = example_dir / 'pnipaam.pdb'

assert fragment_file.exists()
assert structure_file.exists()


In [ ]:
monogrp = MonomerGroup.from_file(fragment_file)
offtop = Topology.from_pdb(structure_file, _custom_substructures=monogrp.monomers)
offtop.visualize()

In [ ]:
from itertools import islice

for atom in islice(offtop.atoms, 19):
    print(atom.metadata['residue_name'], atom.metadata['match_info'])

In [ ]:
was_partitioned = partition(offtop)
assert was_partitioned

resname : str = "pnipaam_TERM1"
display(monogrp.rdmols()[resname][0])
for atom in offtop.atoms:
    if atom.metadata['residue_name'] == resname:
        print(atom.metadata['match_info'], atom.metadata['substructure_query_id'])

In [ ]:
from polymerist.mdtools.openfftools.topology import topology_to_sdf, topology_from_sdf, get_largest_offmol

offmol = get_largest_offmol(offtop)
offmol.visualize(backend='nglview')

offmol.properties['IUPAC name'] = 'poly(N-isopropylacrylamide)'
offmol.properties['First synthesized'] = '1956-12-04'
offmol.properties['Patent No.'] = 'US-2773063-A'
offmol.properties['Patent Holder'] = 'Edward H. Sprecht'
offmol.properties['Description'] = 'A trifling little pile of atoms'

topology_to_sdf(example_dir / 'pnipaam.sdf', offmol.to_topology())